In [3]:
# alright, we just tried a minimal example of a simple neural network
# It's not a good idea to train your neural network from scratch, it's better to use a pre-trained model

# We will use the TorchMD-ET model, and I really recommend for its simplicity and efficiency compared to group-equivariant models
from Geom2Vec.geom2vec.models.torchmd.main_model import create_model, get_args

import torch
import torch.nn as nn

In [5]:
from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import QM9 # We will use the QM9 dataset for this example

# Let's start by loading the dataset

path = '/project/dinner/zpengmei/Geom2Vec/Tutorial/data_sets/QM9'

# QM9 has many labels, let's just pick the first one for now, we can do this with Transform object in PyG

class QM9Transform:
    def __call__(self, data):
        # Select target.
        data.y = data.y[:, 0]
        return data

# Load the dataset, you can't do this on the compute node, you need to do this on the login node for internet access

dataset = QM9(path, transform=QM9Transform()).shuffle()

# Normalize targets to mean = 0 and std = 1.
mean = dataset.data.y.mean(dim=0, keepdim=True)
std = dataset.data.y.std(dim=0, keepdim=True)
dataset.data.y = (dataset.data.y - mean) / std
mean, std = mean[:, 0].item(), std[:, 0].item()

# split the dataset into training, validation and test sets

train_dataset = dataset[:110000]
val_dataset = dataset[110000:120000]
test_dataset = dataset[120000:]

# load your data into the DataLoader

test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

/project/dinner/zpengmei/conda_envs/torch/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:301: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/project/dinner/zpengmei/conda_envs/torch/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [8]:
from torch_scatter import scatter

class Net(nn.Module):

    def __init__(self, hidden_dim, output_dim):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        pt_gnn_args = get_args(
            hidden_channels=hidden_dim, 
            num_layers=9,
            num_rbf=64,
            num_heads=8, 
            cutoff=7.5 # cutoff radius
        )
        self.representation_model = create_model(pt_gnn_args)

        # a simple MLP for regression
        self.task_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, z, pos, batch):

        x_rep,_ , _ = self.representation_model(z, pos, batch)
        x_rep = scatter(x_rep, batch, dim=0, reduce='add')
        out = self.task_head(x_rep)

        return out



In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(hidden_dim=256, output_dim=1).to(device)

checkpoint = torch.load('/project/dinner/zpengmei/subspace_pytorch/pretrain/denoise/denali/logs_models_denali_0.2/tensorboard_logs/ET_l9_256hc_9l_8head_2024-04-12_21-41-17/ET_best.pth')
model.representation_model.load_state_dict(checkpoint, strict=False)

# Now we have a pre-trained model, let's freeze it and train the task head
for param in model.representation_model.parameters():
    param.requires_grad = False


In [21]:
# you can always accelerate your model via automatic mixed precision


# define the optimizer and loss function
from torch.optim import Adam
from torch.nn import L1Loss
from tqdm import tqdm

optimizer = Adam(model.parameters(), lr=1e-3)
criterion = L1Loss()

# define the training loop

def train():
    model.train()

    total_loss = 0
    for data in tqdm(train_loader):
        data = data.to(device)
        optimizer.zero_grad()

        out = model(data.z, data.pos, data.batch) # GNN operates on atomic numbers, positions and batch vector which assigns each atom to a specific molecule
        loss = criterion(out.view(-1), data.y.view(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # show the batch loss in the progress bar
        tqdm.write(f'loss: {loss.item()}')

    return total_loss / len(train_loader)

# define the evaluation loop

@torch.no_grad()
def test(loader):
    model.eval()

    total_loss = 0
    for data in loader:
        data = data.to(device)
        out = model(data.z, data.pos, data.batch)
        total_loss += criterion(out.view(-1), data.y.view(-1)).item()

    return total_loss / len(loader)

In [22]:
# train the model
from tqdm import tqdm

best_val_loss = None
for epoch in tqdm(range(1, 101)):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

    # you can change the frequency of validation and test
    if epoch % 2 == 0:
        val_loss = test(val_loader)
        print(f'Val Loss: {val_loss:.4f}')

        if best_val_loss is None or val_loss <= best_val_loss:
            best_val_loss = val_loss
            # test the accuracy on the test set
            test_loss = test(test_loader)
            print(f'Test Loss: {test_loss:.4f}')


                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
  0%|                                                                                                                  | 0/100 [00:00<?, ?it/s]

loss: 0.5200073719024658
loss: 1.4172673225402832
loss: 0.5361088514328003
loss: 0.9493891596794128


                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
  0%|                                                                                                                  | 0/100 [00:00<?, ?it/s]

loss: 1.0238265991210938
loss: 0.8548645973205566
loss: 0.4784182906150818
loss: 0.5422019958496094
loss: 0.7181023955345154


                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
  0%|                                                                                                                  | 0/100 [00:00<?, ?it/s]

...

                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
  0%|                                                                                                                  | 0/100 [00:12<?, ?it/s]

loss: 0.5613101124763489
loss: 0.48074063658714294
loss: 0.5003455877304077
loss: 0.49599993228912354
loss: 0.5002671480178833


                                                                                                                                               
                                                                                                                                               
  0%|                                                                                                                  | 0/100 [00:13<?, ?it/s]


loss: 0.48379987478256226
loss: 0.46983882784843445


KeyboardInterrupt: 